In [31]:
import yelpapi
import json
import requests
from  pprint import  pprint
import pandas as pd
import numpy as np
import random

counties=pd.read_csv("acs2015_county_data.csv")
counties["county_string"] = counties["County"] + " County, " + counties["State"]
listofcounties = counties["county_string"].tolist()
counties.head()

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_string
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6,"Autauga County, Alabama"
1,1003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5,"Baldwin County, Alabama"
2,1005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6,"Barbour County, Alabama"
3,1007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3,"Bibb County, Alabama"
4,1009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7,"Blount County, Alabama"


In [32]:
zipcodes=pd.read_csv("zip_codes_states.csv")
zipcodes["county_string"] = zipcodes["county"] + " County, " + zipcodes["state"]
listofzipcodes=zipcodes["county_string"].unique().tolist()
zipcodes.head()

,zip_code,city,state,county,county_string
0,36003,Autaugaville,Alabama,Autauga,"Autauga County, Alabama"
1,36006,Billingsley,Alabama,Autauga,"Autauga County, Alabama"
2,36008,Booth,Alabama,Autauga,"Autauga County, Alabama"
3,36051,Marbury,Alabama,Autauga,"Autauga County, Alabama"
4,36066,Prattville,Alabama,Autauga,"Autauga County, Alabama"


In [33]:
#county list and zip list have discrepancy. need to pick counties that have entries in both lists

fullcountylist = list(set(listofcounties) & set(listofzipcodes))
sample_list = random.sample(fullcountylist,25)

In [34]:
#merged_df = zipcodes.merge(counties, left_on="county_string", right_on="county_string", how="left")
#maybe this wan't necessary but it's here if we need it
res_in_county = pd.DataFrame(sample_list)
res_in_county.columns = ["county_string"]
res_in_county["restaurant count"]=0
res_in_county["reviews"]=0
res_in_county["avg score"]=0
res_in_county["num of Italian"]=0
res_in_county["num of Mexican"]=0
res_in_county["num of Chinese"]=0
res_in_county["num of Indian"]=0
res_in_county.head()



,county_string,restaurant count,reviews,avg score,num of Italian,num of Mexican,num of Chinese,num of Indian
0,"Robertson County, Kentucky",0,0,0,0,0,0,0
1,"Red Lake County, Minnesota",0,0,0,0,0,0,0
2,"Crawford County, Arkansas",0,0,0,0,0,0,0
3,"Prince William County, Virginia",0,0,0,0,0,0,0
4,"Wyoming County, West Virginia",0,0,0,0,0,0,0


In [35]:
res_df = pd.DataFrame(columns=["name","zipcode","county_string","rating","review_count","category"])
res_df

,name,zipcode,county_string,rating,review_count,category


In [36]:
r = 0
for c in sample_list:
    county_string = str(c)
    for term in ["Italian","Mexican","Chinese","American (Traditional)","Vegan","Vegetarian","Gluten-Free"]:
        #we are allowed 5000 queries per 24 hours. if we run out, we can create another. till then, use this
        yelpkey="IXRN_1GI8f8J9Kj5O5GciqeyFSWR5RMh41_rADGy1KrtQ5ZABGGZEkWwot5mOka0NlzlNxXPmzYXLWk88oOV0N_YxD1MSW4kQRfJnFrwfHLtKrr2HcsjLo6uWBzAWnYx"
        response = requests.get(url="https://api.yelp.com/v3/businesses/search", 
                                params= {"location": county_string, 
                                         "term": term, "limit":"50"}, 
                                headers={'Authorization': 'bearer %s' % yelpkey})
        responsejson = response.json()
        
        try:
            count=0
            
            for x in range(0,49):
                for y in range(0,len(responsejson["businesses"][x]["categories"])):
                    if ((responsejson["businesses"][x]["categories"][y]["title"]==term) and 
                        (responsejson["businesses"][x]["review_count"] > 0)):
                        
                        zc = int(responsejson["businesses"][x]["location"]["zip_code"])
                        if (zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0]== county_string):
                            
                            print("\n",responsejson["businesses"][x]["name"],
                                  responsejson["businesses"][x]["location"]["zip_code"],
                                  zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0],
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  responsejson["businesses"][x]["categories"][y]["title"])
                            
                            res_df.loc[r]=[responsejson["businesses"][x]["name"],
                                  zc,
                                  county_string,
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  term]
                            count += 1
                            r += 1
        except:
            print("")
    
        print("End of list. Found", count, term, "restaurants in", county_string)
  



End of list. Found 0 Italian restaurants in Robertson County, Kentucky

End of list. Found 0 Mexican restaurants in Robertson County, Kentucky
End of list. Found 0 Chinese restaurants in Robertson County, Kentucky

End of list. Found 0 American (Traditional) restaurants in Robertson County, Kentucky
End of list. Found 0 Vegan restaurants in Robertson County, Kentucky
End of list. Found 0 Vegetarian restaurants in Robertson County, Kentucky
End of list. Found 0 Gluten-Free restaurants in Robertson County, Kentucky

End of list. Found 0 Italian restaurants in Red Lake County, Minnesota

End of list. Found 0 Mexican restaurants in Red Lake County, Minnesota

End of list. Found 0 Chinese restaurants in Red Lake County, Minnesota

 Joe DiMaggio's Sports Cafe 56750 Red Lake County, Minnesota 3.5 2 American (Traditional)

End of list. Found 1 American (Traditional) restaurants in Red Lake County, Minnesota

End of list. Found 0 Vegan restaurants in Red Lake County, Minnesota

End of list. Fo


 New China Restaurant 31546 Wayne County, Georgia 3.5 9 Chinese

 Peking Gourmet 31545 Wayne County, Georgia 3.0 3 Chinese

 Grand Buffet 31545 Wayne County, Georgia 3.0 1 Chinese
End of list. Found 3 Chinese restaurants in Wayne County, Georgia

 Hungry HillBilly's Grill & BBQ 31546 Wayne County, Georgia 4.5 21 American (Traditional)

 That Place BBQ 31546 Wayne County, Georgia 4.0 5 American (Traditional)

 The Cafeteria at Wayne Memorial Hospital 31545 Wayne County, Georgia 5.0 1 American (Traditional)

 Sybil's Family Restaurant 31545 Wayne County, Georgia 3.5 16 American (Traditional)

 Alec's Sports Bar and Grill 31545 Wayne County, Georgia 2.0 3 American (Traditional)

 Waffle House 31545 Wayne County, Georgia 2.5 2 American (Traditional)

 Damon's Famous Fingers & Wings 31545 Wayne County, Georgia 3.5 22 American (Traditional)
End of list. Found 7 American (Traditional) restaurants in Wayne County, Georgia
End of list. Found 0 Vegan restaurants in Wayne County, Georgia
End of 


 Danny Haskell's Pub 53150 Waukesha County, Wisconsin 4.5 8 American (Traditional)

 The Station Pub & Grill 53072 Waukesha County, Wisconsin 5.0 2 American (Traditional)

 Revere's Wells Street Tavern 53018 Waukesha County, Wisconsin 3.0 104 American (Traditional)

 Coppers Pub & Grill 53072 Waukesha County, Wisconsin 3.0 7 American (Traditional)

 The Spot on broadway 53186 Waukesha County, Wisconsin 3.0 8 American (Traditional)

 Maxim's at Oconomowoc Depot 53066 Waukesha County, Wisconsin 3.5 54 American (Traditional)

 Dickey's Barbecue Pit 53072 Waukesha County, Wisconsin 2.0 42 American (Traditional)

 Matty's Bar & Grille 53151 Waukesha County, Wisconsin 4.0 126 American (Traditional)

 Chili's 53189 Waukesha County, Wisconsin 3.5 15 American (Traditional)
End of list. Found 46 American (Traditional) restaurants in Waukesha County, Wisconsin
End of list. Found 0 Vegan restaurants in Waukesha County, Wisconsin

 Cafe Manna 53045 Waukesha County, Wisconsin 4.0 192 Vegetarian

 G

 Markham Street Grill & Pub 72211 Pulaski County, Arkansas 3.0 13 American (Traditional)

 Movie Tavern - Little Rock 72210 Pulaski County, Arkansas 3.0 5 American (Traditional)

End of list. Found 30 American (Traditional) restaurants in Pulaski County, Arkansas

 Viva Vegan 72204 Pulaski County, Arkansas 4.5 8 Vegan

 Izzy's 72223 Pulaski County, Arkansas 4.0 54 Vegan

 The Fold: Botanas & Bar 72202 Pulaski County, Arkansas 4.0 139 Vegan
End of list. Found 3 Vegan restaurants in Pulaski County, Arkansas

 Big Orange 72205 Pulaski County, Arkansas 4.5 273 Vegetarian

 Brave New Restaurant 72202 Pulaski County, Arkansas 4.0 108 Vegetarian

 Taziki's Mediterranean Cafe 72116 Pulaski County, Arkansas 4.0 26 Vegetarian

 Big Orange 72223 Pulaski County, Arkansas 4.0 176 Vegetarian

 4square Cafe & Gifts 72201 Pulaski County, Arkansas 4.0 31 Vegetarian

 Rosalia's Family Bakery 72205 Pulaski County, Arkansas 4.0 10 Vegetarian

 All Aboard Restaurant And Grill 72207 Pulaski County, Arkansas


 Gasthof Amish Village 47558 Daviess County, Indiana 4.0 23 American (Traditional)

 Mulligans Restaurant & Bar 47501 Daviess County, Indiana 4.5 3 American (Traditional)

 The Hydrant Bar & Grill 47501 Daviess County, Indiana 4.0 11 American (Traditional)

 Red Bones 47558 Daviess County, Indiana 3.5 2 American (Traditional)

 Benjy's 47501 Daviess County, Indiana 4.5 4 American (Traditional)
End of list. Found 5 American (Traditional) restaurants in Daviess County, Indiana
End of list. Found 0 Vegan restaurants in Daviess County, Indiana
End of list. Found 0 Vegetarian restaurants in Daviess County, Indiana
End of list. Found 0 Gluten-Free restaurants in Daviess County, Indiana

End of list. Found 0 Italian restaurants in Franklin County, Washington

 El Antojito Mexican Restaurant 99301 Franklin County, Washington 4.5 36 Mexican

 La Herradura 99326 Franklin County, Washington 4.5 7 Mexican

 Mexico Lindo Y Que Rico 99301 Franklin County, Washington 4.0 38 Mexican

 Mi Lindo Nayari


 Martino's Lounge 97520 Jackson County, Oregon 3.5 52 Italian

 Northwest Pizza 97520 Jackson County, Oregon 4.0 72 Italian

 Pizza Hut 97502 Jackson County, Oregon 2.0 8 Italian

 Pizza Hut 97501 Jackson County, Oregon 4.0 5 Italian

 Villa Fresh Italian Kitchen 97501 Jackson County, Oregon 3.5 3 Italian

 Pizza Hut 97504 Jackson County, Oregon 1.5 13 Italian
End of list. Found 22 Italian restaurants in Jackson County, Oregon

 El Molcajete Mexican Grill 97501 Jackson County, Oregon 4.5 204 Mexican

 Juan's El Caporal 97524 Jackson County, Oregon 4.0 24 Mexican

 La Tapatia Restaurant 97535 Jackson County, Oregon 4.5 204 Mexican

 Trail Market Tacos 97541 Jackson County, Oregon 5.0 12 Mexican

 Mazatlan Grill 97502 Jackson County, Oregon 4.0 102 Mexican

 Casa Latina Carniceria y Taqueria 97503 Jackson County, Oregon 4.0 21 Mexican

 Taqueria Picaro 97520 Jackson County, Oregon 4.0 119 Mexican

 Miguel's Guadalajara Restaurant 97539 Jackson County, Oregon 2.5 60 Mexican

 Fiesta Taqu


 NW Raw Organic Juice Bar 97520 Jackson County, Oregon 4.5 80 Vegan
End of list. Found 1 Vegan restaurants in Jackson County, Oregon

 Sauce Whole Food Cafe 97520 Jackson County, Oregon 4.0 280 Vegetarian

 Burger Spot 97501 Jackson County, Oregon 4.5 58 Vegetarian

 Essential Wellness 97520 Jackson County, Oregon 4.5 8 Vegetarian

 Tacopancho Truck 97501 Jackson County, Oregon 4.5 8 Vegetarian

 Aloha Grill 97520 Jackson County, Oregon 4.0 18 Vegetarian

 Asian Grill 97504 Jackson County, Oregon 3.0 15 Vegetarian
End of list. Found 6 Vegetarian restaurants in Jackson County, Oregon

 Sauce Whole Food Cafe 97520 Jackson County, Oregon 4.0 280 Gluten-Free

 Spoons 97502 Jackson County, Oregon 4.5 13 Gluten-Free

 TonTon's Artisan Affections 97540 Jackson County, Oregon 5.0 7 Gluten-Free

 NW Raw Organic Juice Bar 97520 Jackson County, Oregon 4.5 80 Gluten-Free
End of list. Found 4 Gluten-Free restaurants in Jackson County, Oregon
End of list. Found 0 Italian restaurants in Clay County,


 Hellbender Burritos 26260 Tucker County, West Virginia 4.0 148 Mexican
End of list. Found 1 Mexican restaurants in Tucker County, West Virginia

End of list. Found 0 Chinese restaurants in Tucker County, West Virginia

 The Sawmill Restaurant 26260 Tucker County, West Virginia 4.0 32 American (Traditional)

 Deerfield Restaurant and Pub 26260 Tucker County, West Virginia 4.0 20 American (Traditional)

 Wicked Wilderness 26260 Tucker County, West Virginia 4.0 7 American (Traditional)

 Amelia's Restaurant 26260 Tucker County, West Virginia 4.0 15 American (Traditional)

 Little Andi's Resturaunt 26287 Tucker County, West Virginia 3.5 6 American (Traditional)

 Big John's Family Fixin's 26260 Tucker County, West Virginia 2.0 28 American (Traditional)
End of list. Found 6 American (Traditional) restaurants in Tucker County, West Virginia
End of list. Found 0 Vegan restaurants in Tucker County, West Virginia

 Flying Pigs 26292 Tucker County, West Virginia 3.5 29 Vegetarian
End of list. 

In [37]:
res_df

,name,zipcode,county_string,rating,review_count,category
0,Joe DiMaggio's Sports Cafe,56750,"Red Lake County, Minnesota",3.5,2,American (Traditional)
1,Frank's Italian Restaurant,72956,"Crawford County, Arkansas",4.0,56,Italian
2,Pizza Hut,72921,"Crawford County, Arkansas",3.0,4,Italian
3,Pizza Hut,72956,"Crawford County, Arkansas",1.0,4,Italian
4,El Trio,72921,"Crawford County, Arkansas",4.5,16,Mexican
5,La Fiesta,72921,"Crawford County, Arkansas",4.0,12,Mexican
6,La Huerta,72956,"Crawford County, Arkansas",4.5,8,Mexican
7,Las Fresas,72956,"Crawford County, Arkansas",3.0,10,Mexican
8,Pancho's Mexican Grill,72956,"Crawford County, Arkansas",4.0,1,Mexican
9,La Fiesta Mexican Restaurant,72956,"Crawford County, Arkansas",3.5,15,Mexican


In [38]:
res_df["category"].value_counts()

American (Traditional)    242
Mexican                   187
Chinese                   116
Italian                    82
Gluten-Free                22
Vegetarian                 18
Vegan                       6
Name: category, dtype: int64

In [39]:
responsejson["businesses"]

[{'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'},
   {'alias': 'bakeries', 'title': 'Bakeries'},
   {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
  'coordinates': {'latitude': 39.1490922184564, 'longitude': -79.498736308658},
  'display_phone': '(304) 463-4455',
  'distance': 6618.574236574174,
  'id': 'tiptop-thomas-3',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/4K6ofUF-Yf81Tg0iXRsHWg/o.jpg',
  'is_closed': False,
  'location': {'address1': '216 E Ave',
   'address2': '',
   'address3': '',
   'city': 'Thomas',
   'country': 'US',
   'display_address': ['216 E Ave', 'Thomas, WV 26292'],
   'state': 'WV',
   'zip_code': '26292'},
  'name': 'TipTop',
  'phone': '+13044634455',
  'price': '$$',
  'rating': 4.5,
  'review_count': 66,
  'transactions': [],
  'url': 'https://www.yelp.com/biz/tiptop-thomas-3?adjust_creative=G_DkR7WiUJ0P-HF5MN1vmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=G_DkR7WiUJ0P-HF5MN1vmg'},
 {'categories': [

In [8]:
yelpkey="IXRN_1GI8f8J9Kj5O5GciqeyFSWR5RMh41_rADGy1KrtQ5ZABGGZEkWwot5mOka0NlzlNxXPmzYXLWk88oOV0N_YxD1MSW4kQRfJnFrwfHLtKrr2HcsjLo6uWBzAWnYx"
response = requests.get(url="https://api.yelp.com/v3/transactions/delivery/search", 
            params= {"location": "28105","term": "Mexican", "limit":"50"}, 
            headers={'Authorization': 'bearer %s' % yelpkey})
pprint(response.json())

{'businesses': [], 'total': 0}
